# Analysis of Sliding Window method with 76. Minimum Window Substring (hard)

Question Link: https://leetcode.com/problems/minimum-window-substring/

## Step 1: Analysis of Bruteforce Solution

The question requires us to find out the shortest substring in a given string "s" that contains all characters of another given string "t".

Initially the question does not seem it is a "hard" question, but the question is quite tricky in terms of reducing its time complexity, and even calculating the time complexity itself is a tricky thing to do.

First let's look at the bruteforce solution

The bruteforce solution is quite straightforward. If we can come up and visit every single substring, and when visiting each substring, determine whether the substring contains the characters in string "t." Through doing that, keep a record for the shortest substring that satisfies the condition and finally we will have the shortest substring that contains all of t's characters.

The algorithm is like this:
```
for i in range(len(s)):
    for j in range(j+1, len(s)):
        determine whether s[i:j] contains all characters in "t"
```        
For the part to determin whether the substring contains all characters in t, we cannot simply use a nested loop to go through the substring. Instead we will need to take care of the situation where t has multiple occurrences of the same character. For a bruteforce way of determining whether all characters in t are in substring ss, we can use the following algorithm.
```
for c in t:
    if ss.count(c) < t.count(c):
        ss does not contain all t's characters
ss contains all t's characters
```
Of course, the above is the most plain and easiest to understand. It is easy to make some optimization to the code so that the efficiency can be improved. But for now, let's accept this easy-to-understand bruteforce solution.

The code is as following.

In [1]:
# Burteforce solution.
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        result = None
        for i in range(len(s)):
            for j in range(i+1, len(s)+1):
                if self.t_in_ss(s[i:j], t):
                    result = s[i:j] if result is None or len(result) > len(s[i:j]) else result
        return result if result else ""

    def t_in_ss(self, ss, tg):
        for c in tg:
            if tg.count(c) > ss.count(c):
                return False
        return True

In [4]:
# A few test cases
cases = [["ADOBECODEBANC", "ABC"],
         ["a", "a"], ["a", 'aa']]

for str_s, str_t in cases:
    print(Solution().minWindow(str_s, str_t))

BANC
a



Because of the high time complexity of the solution, the code won't pass the test on Leetcode. A further optimization to reduce the time complexity would be needed. But first let's understand the bruteforce solution's time complexity in terms of its big O representation.

First Let's introduce the addition and multiplication laws of Big O, which are simply:
- O(f(n)) + O(g(n)) = O(f(n)+g(n))
- O(f(n)) \* O(g(n)) = O(f(n)\*g(n))

Now to understand the time complexity of the bruteforce solution, we check see that in the outer for loops we have O(n) * O(n) = O(n\*n).

But it's a bit tricky to find out the helper function's time complexity because the count method in the str type is unknown. After searching for it, the closest answer I can find is here. 
    https://stackoverflow.com/questions/35855748/whats-the-computational-cost-of-count-operation-on-strings-python
    
It states the time complexity for searching for one character in another string is O(n). Let's take this as true, and we can see that two count methods are invoked in the helper function and one outer loop is used. Let's say len(t) is m, and len(s) is n. 

For the outer loop and the t.count(c) call, the time complexity is O(m). The ss.count(c) call is generally O(n).

Combined with the two next for loops in the minWindow function, the time complexity of the solution should be:
> O(n\*n\*m\*(m+n))

A pretty high time complexity.


## Step 2: Sliding Window Method

To try to reduce the steps we have to take to find out the shortest substring that satisfies the condition, we have to look deeper into the bruteforce solution and find out what are the unneccessary steps that we can skip.

To visit all possibel substrings in the bruteforce method, the solution used a nest for loop, in which it "pins" the left position and expand the right position in the inner for loop to go over all possibilities. Once an inner for loop finishes, meaning that the right position reaches the last position of the string s, the left position is moved 1 position forward, and the right position is reset to the next position of the new left position.

The first thing we can do is that instead of letting the right position go all the way to the end of string s, we can stop the expansion once, for a "pinned" left position, a shortest substring containing all characters in string t is found. The reason for stopping the right position from moving is obvious, once the shortest solution for a fixed left position is found, we can end the inner for loop, because all substrings in the following exploration in the same inner for loop will be a longer solution than the one found, hence won't be accepted.

This is just a minor change to the bruteforce solution and will not reduce the time complexity.


In [6]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        result = None
        for i in range(len(s)):
            for j in range(i+1, len(s)+1):
                if self.t_in_ss(s[i:j], t):
                    result = s[i:j] if result is None or len(result) > len(s[i:j]) else result
                    break
        return result if result else ""

    def t_in_ss(self, ss, tg):
        for c in tg:
            if tg.count(c) > ss.count(c):
                return False
        return True

In [7]:
# A few test cases
cases = [["ADOBECODEBANC", "ABC"],
         ["a", "a"], ["a", 'aa']]

for str_s, str_t in cases:
    print(Solution().minWindow(str_s, str_t))

BANC
a



In the above optimized solution, we perform the reset earlier than after completing the whole process of exploring all the substrings.

However, this method still resets both left and right positions after each inner for loop so that we can start with the next left prosition. Let's consider a method that we don't need to reset both positions but only the left.

That means when we start the next loop, we don't move the right position but move the left position one step forward. And if this substring between the left and right positions still contains all characters of string t, It means we have a one-character shorter answer than the one we found previously. And we can then continue this process, that is, keep moving the left position one step forward and determine whether the substring between left and right positions satisfies our condition. Eventually we will find out the shortest substring with that fixed right position.

But this shortest substring is only the shortest with that right position but not necessarily the shortest substring that we want. For that, we would need to further explore other possibilitie. 

Starting from where we left off, if we have already found the shortest substring with that fixed right position, what are the next steps for other possibilities? Just imagine how the exploration process will stop. We have reached the last left position so that the corresponding substring is the shortest with the particular right position, but at this time, we don't know we have found it; we still move the left position one step forward and then at this step, the substring we get to no longer contains all characters from string t. Then in the next step, we should not forward the left position again because if this substring does not satisfy our need, the next substring will not either. Here we should move the right position again if it has not reached the last character of the string s.

How should we move the right position? Basically we have two options that make sense.
1. set the right position next to the current left position.
2. move the right position forward.

For the first option, it is not a good choice, because if we set the right position next to the left position and move forward again, we would have to explore some substrings we know will not contain all characters of string t, that is, substrings with the current left position and a right position between the one next to the current left position and the right position before the reset. Hence the only logical move is 2, move the right position forward.

And then we can explore more substrings by moving the right position foreward until we have found another substring that contains all characters of string t. After that we can start moving the left postion forward again until we find that the substring does not satisfy the condition anymore. And then move the right position forward ...

This moving the right position forward and then the left positon algorithm is called the sliding window method. It's named after the moving right and left sides of a "window." It is quite similar to a window sliding through a string.

Let's look at the solution code using the sliding window method.


In [8]:
# Sliding window method.
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        result = None
        left = right = 0
        while right <= len(s):
            if len(s[left:right])<len(t) or not self.t_in_ss(s[left:right], t):
                right += 1
            else:
                result = s[left:right] if result is None or len(result) > len(s[left:right]) else result
                left += 1
        return result if result else ""

    def t_in_ss(self, ss, tg):
        for c in tg:
            if tg.count(c) > ss.count(c):
                return False
        return True

In [9]:
# A few test cases
cases = [["ADOBECODEBANC", "ABC"],
         ["a", "a"], ["a", 'aa']]

for str_s, str_t in cases:
    print(Solution().minWindow(str_s, str_t))

BANC
a



Very unfortunately, this sliding window method still won't pass the test on Leecode. As for the sliding window application, the original nested for loop is gone and replaced with a while loop, hence time complexity has been reduced from O(n\*n) to O(n). However the helper function t_in_ss's time complexity is still the same. The time complexity for the whole solution is O(n\*m\*(m+n)), which is still pretty high.

Clearly the next step is to rewrite the t_in_ss helper function.


## Step 3: Space for Time

One very classic way for improving the time complexity of an algorithm is to trade space with time. Usually it means using a "record" to keep track of the history so that the operations (usually comparisons) don't have to be done again. In this question, we can also apply the same technique to improve the t_in_ss helper function's time complexity.

In the current helper function, we try to search every character in string t in substring ss. This operations happens over and over again, even when only the left or the right position is put to the next position. There is a lot of wasted resource in this process.

To reduce the waste, we can simply maintain the status of the current window and that of the target, then we can compare the two to find out whether the substring in our window contains all characters of string t. When the window is sliding, we can change the status of the current window according to what is removed and what is added to it.



To do the above, we will need to maintain two counters: one is for the characters in t, and the other is for the characters inside the window. When the right position moves to the next position, we include the new character in the window counter; and then the left postion moves to the next position, we exclude the old character in the window counter.

And to compare whether a window contains the same or more counts of the characters in string t, we use another variable is_valid to keep track of how many characters in t have enough count inside the window. To do so, every time a new character is included in the window, we will compare the count of the character in window with that in t: if they are the same, we increase is_valid by one, meaning this character has enough presence in the window. And as long as the is_valid has the same value as the number of DISTINCT characters in t, we know that the window contains a solution. And when a character is excluded from the window, we can kind of revert the above process.

Let's look at the code.

In [10]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        wd, tg, is_valid = {}, {}, 0
        l = r = is_valid = 0
        result = None
        
        for c in t:
            tg[c] = tg.get(c, 0) + 1

        while r<len(s) or is_valid==len(tg): 
            if is_valid<len(tg) and r<len(s): # expanding the window
                if s[r] in tg: # only record the count of characters in string t
                    wd[s[r]] = wd.get(s[r], 0) + 1 # increase count of character
                    if wd[s[r]] == tg[s[r]]: is_valid += 1 # when count in window equals to the count in t,
                                                           # one more character satisfies the condition
                r += 1 # move the right border of the window
            elif is_valid == len(tg): # dwindling the window
                result = s[l:r] if not result or len(result) > len(s[l:r]) else result # update the shortest substring
                if s[l] in tg:
                    wd[s[l]] -= 1 # decrease count of character
                    if wd[s[l]] < tg[s[l]]: is_valid -= 1 # when count in window is less than the count in t,
                                                          # one less character satisfies the conditon
                l += 1 # move the left border of the window
        return result if result else ""


There are a lot of details in the above solution that we would need to take care. Please check out the code comments.

The while loop condition is a bit tricky and the conditions here are not the only correct conditions. Some other solutions even put a True here and break the while loop inside it.

We already established what is done inside the while loop:
1. expanding the window when the window does not contain a solution.
2. shrinking the window when the window still contains the solution.

For expansion of the window, we can still expand the window if the right border of the window is not at the end of the string s. For the shrinking of the window, the left border of the window cannot go past the right border.

The tricky bit of the process is that even when the right border of the window reaches the end of the string s, we may still need to shrink the window to find the shortest substring. That is why the while loop condition is set as a union of 1) either right is not at the end of string s, 2) the window still contains a solution even when the right border reaches the end. 

In [11]:
# A few test cases
cases = [["ADOBECODEBANC", "ABC"],
         ["a", "a"], ["a", 'aa']]

for str_s, str_t in cases:
    print(Solution().minWindow(str_s, str_t))

BANC
a



One last thing let's look at the time complexity of this solution.

The last solution creates the counter with O(m) time, where m is the length of string t. And in the while loop, the time complexity is O(2\*n) = O(n). Hence, the solution's time complexity is O(m) + O(n) = O(m+n).

## Similar Questions

3. Longest Substring Without Repeating Character: https://leetcode.com/problems/longest-substring-without-repeating-characters/
